# Лабораторная работа 2, вариант1. Збруев Денис

### Подключим библиотеки и настроим датасет MNIST

In [1]:
from sklearn.cluster import KMeans, DBSCAN, Birch 
from sklearn import metrics
from torchvision import datasets, transforms
import torch
import torch.utils.data as data_utils
import numpy as np

from torchvision.datasets import MNIST

#загружаем набор данных MNIST
transform = transforms.Compose([
transforms.ToTensor(),
transforms.Normalize((0.1307,), (0.3081,))  
])

train_set = MNIST(root='./data', train=True, download=True, transform=transform) #датасет, на котором "тренируем" нейронку
test_set = MNIST(root='./data', train=False, download=True, transform=transform) #на этом датасете проверям, насколько хорошо обучили нейронку (через расчет ARI)

train_data = train_set.data.reshape((-1, 28 * 28)) #делаем картинки одного размера
test_data = test_set.data.reshape((-1, 28 * 28))

train_data = train_data.float() / 255.0   #алгоритм так быстрее работает
test_data = test_data.float() / 255.0

### Произведём k-means класстеризацию и высчитаем значение ARI для k-means:

In [5]:
kmeans = KMeans(n_clusters=10, random_state=12).fit(train_data) 
#Получение прогнозируемых меток
kmeans_predicate_labels = kmeans.predict(test_data) 
#Рассчитываем скорректированный индекс RAND между прогнозируемыми метками и истинными метками
true_labels = test_set.targets
kmeans_ari = metrics.adjusted_rand_score(true_labels, kmeans_predicate_labels)
print("kmeans Adjusted Rand Index:", kmeans_ari)

C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


kmeans Adjusted Rand Index: 0.4049002590872505


### Произведём DBSCAN класстеризацию и высчитаем значение ARI для DBSCAN:

In [7]:
dbscan = DBSCAN(eps=5.2, min_samples=6) #на 5.2 оптимален, min_samples- это как раз n из лекции. 6 - оптимален (ari выше) #через чат подобрать оптимальные параметры утром
dbscan.fit(train_data)
dbscan_predicated_labels = dbscan.fit_predict(test_data) 
true_labels = test_set.targets
dbscan_ari = metrics.adjusted_rand_score(true_labels, dbscan_predicated_labels)
print("DBSCAN  Adjusted Rand Index:",dbscan_ari)

DBSCAN  Adjusted Rand Index: 0.1089627571656831


### Произведём BIRCH класстеризацию и высчитаем значение ARI для BIRCH:

In [8]:
birch = Birch(branching_factor = 40, n_clusters = None, threshold = 7.0).fit(train_data)  #branching_factor=40 оптимален, на 7.0 оптимально threshold=0.3, n_clusters=10).fit(train_data)#preference=-50, damping=0.5).fit(train_data)   #через чат подобрать оптимальные параметры утром
birch_predicated_labels = birch.predict(test_data) 
true_labels = test_set.targets
birch_ari = metrics.adjusted_rand_score(true_labels, birch_predicated_labels)
print("Birch  Adjusted Rand Index:",birch_ari)

Birch  Adjusted Rand Index: 0.32461455140691714


### Вывод: рассмотрены 3 алгоритма. Хуже всего себя показал алгоритм DBSCAN (минимальное значение ARI).  Это может быть связано с тем, что DBSCAN эффективно выделяет кластеры с константной плотностью. Изображение цифр не обладает константной плотностью, что приводит к ошибкам кластеризации. Кроме того, еще одна возможная причина плохой работы DBSCAN - изображения MNIST имеют много шума.  На втором месте оказался алгоритм Birch, а на первом месте - Kmeans. Вероятная причина победы Kmeans связана с тем, что он хорошо работает с данными, которые имеют ярко выраженные кластеры (кластеры в MNIST неплохо выражены). Это позволяет kmeans разделить изображения на кластеры с высокой точностью  (относительно других рассмотренных в этой лабораторной работе алгоритмов кластеризации).